In [3]:
import threading
import time
import emoji
import re
import random
import requests
import pandas as pd
from bs4 import BeautifulSoup
#流程控制
def random_headers():
    headers_list = [{'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0'},{'User-agent':' Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'},{'User-agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.106 Safari/537.36 OPR/38.0.2220.41'},{'User-agent':'Mozilla/5.0 (Linux; U; Android 4.0.3; de-ch; HTC Sensation Build/IML74K) AppleWebKit/534.30 (KHTML, like Gecko) Version/4.0 Mobile Safari/534.30'},{'User-agent':'Mozilla/5.0 (compatible; MSIE 9.0; Windows Phone OS 7.5; Trident/5.0; IEMobile/9.0)'}]
    headers=random.choice(headers_list)
    print(headers)
    return headers
    

    
def get_soup(url,headers):
    proxy_list=[{'https':'59.124.224.180:3128','http':'59.124.224.180:3128'},{'https':'113.196.140.162:8888','http':'113.196.140.162:8888'}]
    proxy_list2={'http':'http://220.135.165.38:8080','https':'220.135.168.38:8080'}

    ss=requests.session()
    res=ss.get(url,headers=headers)
  
    
    res_status_code=res.status_code
    print(res_status_code)
    
    if res_status_code!=200 :
        time.sleep(2)
        print("進入proxy狀態")
        proxy=random.choice(proxy_list)
        change_count+=1
        res = requests.get(url ,proxies=proxy ,headers=headers)
        soup=BeautifulSoup(res.text,'html.parser')
   
    soup=BeautifulSoup(res.text,'html.parser')
    return soup

def preprocess (x):
    x = emoji.get_emoji_regexp().sub(r'', x)
    x = re.sub(r'[［(（【{][^】)}）］]*[】)}）］]', '', x)
    x = x.strip().replace(" ", "")
    x = re.sub(r'[~\-。]', "", x) 
    return x

    

#random_headers()


#初始變數

headers = {'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0'}
url = 'https://icook.tw/categories?ref=icook-footer'
get_soup(url,headers)
soup=get_soup(url,headers)

#tag_list = soup.select('a.categories-all-child-link')
#section_ls = []
#for tag in tag_list:
#    section_ls.append(tag['href'])

#your_section = section_ls[len(section_ls)//2:]

#your_section=['/categories/39' , '/categories/64', '/categories/63', '/categories/216', '/categories/350', '/categories/219', '/categories/20', '/categories/463', '/categories/599', '/categories/462', '/categories/148', '/categories/27', '/categories/147', '/categories/210', '/categories/206', '/categories/77', '/categories/498', '/categories/453', '/categories/495', '/categories/29', '/categories/30', '/categories/455', '/categories/456', '/categories/13', '/categories/449', '/categories/26', '/categories/25', '/categories/110', '/categories/417', '/categories/458', '/categories/459', '/categories/600', '/categories/612', '/categories/613', '/categories/614', '/categories/616', '/categories/211', '/categories/185', '/categories/52', '/categories/71', '/categories/137', '/categories/49', '/categories/50', '/categories/602', '/categories/607']
#your_section=['/categories/600', '/categories/614', '/categories/417', '/categories/25', '/categories/456', '/categories/459', '/categories/613', '/categories/219', '/categories/185', '/categories/607', '/categories/30', '/categories/350', '/categories/498', '/categories/616', '/categories/52', '/categories/64', '/categories/462', '/categories/599', '/categories/137', '/categories/63', '/categories/210', '/categories/211', '/categories/458', '/categories/49', '/categories/71', '/categories/453', '/categories/13', '/categories/110', '/categories/602', '/categories/77', '/categories/20', '/categories/463', '/categories/455', '/categories/495', '/categories/216', '/categories/449', '/categories/148', '/categories/50', '/categories/26', '/categories/29', '/categories/147', '/categories/206', '/categories/39', '/categories/612', '/categories/27']

#your_section1=your_section[0:9]
your_section1=['/categories/49', '/categories/71','/categories/453','/categories/458']



def job(x):
    headers = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'}
    domain = 'https://icook.tw'
    url_categories = x
    page = 1
    vip_count=0
    c = 0
    url = domain + url_categories
    df = pd.DataFrame(columns = ['Recipe Name','Author','Recipe URL','Recipe Picture','Servings','Time','Ingredients',\
                                 'Unit','Recipe Like','Publish Date','Views Number','Recipe Steps'])
    headers=random_headers()
    soup=get_soup(url,headers)
    section_number = url.split('/')[2]
    total_amount = soup.select('title')[0].text
    total_amount = int(''.join(re.findall("\d+", total_amount)))
    max_page = 0
    if total_amount % 18 == 0:
        max_page = total_amount // 18
    else:
        max_page = (total_amount // 18) + 1
    
    while(page <= max_page):
        page_index = f'?page={page}'
        url = domain + url + page_index
        print(url)
        headers= random_headers()
        soup=get_soup(url,headers)
        food_list = soup.select('a.browse-recipe-touch-link')
        for food in food_list:
            headers= random_headers()
            soup=get_soup(url,headers)
            
            #recipe_section = soup_recipe.select('a.recipe-breadcrumb-item-link')[1].text.strip()
            #recipe_title = "".join((soup_recipe.select('h1.title')[0].text).split())
            #recipe_title = preprocess(recipe_title)
            recipe_title = soup.select('h1.title')[0].text
            recipe_title = preprocess(recipe_title)
            check_vip=soup.select('h1.title')[0].select('span.badge-vip')
            if len(check_vip)<=0:
                pass
                #print("is not vip data, data will be save")
            else:
                print("this is vip data,data will not save")
                vip_count+=1
                continue
            
            
            
            recipe_author = soup.select('a.author-name-link')[0].text
            recipe_pic_url = soup.select("img.main-pic")[0]['src']
            
        
            if len(soup.select('span.stat-content')) == 2:
                recipe_like = 0
            else:
                if '萬' in soup.select('span.stat-content')[0].text:
                    recipe_like = int(soup.select('span.stat-content')[0].text[:-5].replace('.','')+"000")
                else:
                    recipe_like = int(soup.select('span.stat-content')[0].text[:-3].replace(',',''))
        
            if not soup.select('div.servings-info.info-block') and not soup.select('div.time-info.info-block'):
                recipe_servings = 0
                recipe_time = 0
            elif not soup.select('div.time-info.info-block'):
                recipe_servings = int(soup.select('span.num')[0].text)
                recipe_time = 0
            elif  not soup.select('div.servings-info.info-block'): 
                recipe_servings = 0
                recipe_time = int(soup.select('span.num')[0].text)
            else: 
                recipe_servings = int(soup.select('span.num')[0].text)
                recipe_time = int(soup.select('span.num')[1].text)

            recipe_ingredient = ""
            for ingredient in soup.select('.ingredient-search'):
                if ingredient.text == "請參考圖片說明":
                    continue
                else:
                    recipe_ingredient += ingredient.text + " " 
            recipe_ingredient = emoji.get_emoji_regexp().sub(r'', recipe_ingredient)
        
            recipe_unit = ""
            for unit in soup.select('div.ingredient-unit'):
                if unit.text == "内文有":
                    continue
                else:
                    recipe_unit += unit.text + " "

            recipe_steps = ""
            counter = 1
            for steps in soup.select('div.step-instruction-content'):
                recipe_steps += str(counter) + " " + steps.text + "\n"
                recipe_steps = emoji.get_emoji_regexp().sub(r'', recipe_steps)
                counter += 1
            publish_date = soup.select('span.meta-content')
            if len (publish_date)<=0:
                print("publish_date is na")
                publish_date="na"
            else:
                publish_date = publish_date[0].text[:-3]
        
            views = soup.select('span.meta-content')
            if len(views)<=0:
                print("view is na")
                views_number=0
                views="0"
            else:
                views=views[1].text[:-3]
                if '萬' in views:
                    views_number = views[:-2]
                    views_number = int(views_number.replace('.','')+"000")
                else:
                    views_number = int(views.replace(',',''))
        
        
            
            #print(recipe_section)    
            print(recipe_url)
            print(recipe_title)
            #print(recipe_author)
            #print(recipe_pic_url)
            #print(recipe_like)
            #print(recipe_servings)
            #print(recipe_time)
            #print(recipe_ingredient)
            #print(recipe_unit)
            #print(recipe_steps)
            #print(publish_date)
            #print(views)
            print("----")
        
            ls = [recipe_title,recipe_author,recipe_url,recipe_pic_url,recipe_servings,recipe_time,recipe_ingredient,\
                  recipe_unit,recipe_like,publish_date,views_number,recipe_steps]
            df.loc[c] = ls
            c += 1
    
            time.sleep(random.randint(1, 4))
        
        
    
        page += 1
    df.to_csv(f'./{section_number}.csv', index=False ,encoding='utf-8-sig')
    print("有多少{}vip資料".format(vip_count))


threads = list()
starttime = time.time()
#a = ['/categories/347','/categories/346']
for element in your_section1:
    threads.append(threading.Thread(target=job, args=(element,)))
for thread in threads:    
    thread.start()
for thread in threads:
    thread.join()
print('Multithread took {} seconds'.format(time.time() - starttime))



200
200
{'User-agent': 'Mozilla/5.0 (compatible; MSIE 9.0; Windows Phone OS 7.5; Trident/5.0; IEMobile/9.0)'}{'User-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.106 Safari/537.36 OPR/38.0.2220.41'}

200
200
200
200
https://icook.twhttps://icook.tw/categories/612?page=1
{'User-agent': ' Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}
https://icook.twhttps://icook.tw/categories/206?page=1
{'User-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.106 Safari/537.36 OPR/38.0.2220.41'}


Exception in thread Thread-22:
Traceback (most recent call last):
  File "E:\python\anaconda3\envs\python_study\lib\site-packages\urllib3\connection.py", line 157, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw
  File "E:\python\anaconda3\envs\python_study\lib\site-packages\urllib3\util\connection.py", line 61, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "E:\python\anaconda3\envs\python_study\lib\socket.py", line 752, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 11004] getaddrinfo failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "E:\python\anaconda3\envs\python_study\lib\site-packages\urllib3\connectionpool.py", line 672, in urlopen
    chunked=chunked,
  File "E:\python\anaconda3\envs\python_study\lib\site-packages\urllib3\connectionpool.py", line 376, in _make_requ

Multithread took 5.656000137329102 seconds


Exception in thread Thread-23:
Traceback (most recent call last):
  File "E:\python\anaconda3\envs\python_study\lib\site-packages\urllib3\connection.py", line 157, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw
  File "E:\python\anaconda3\envs\python_study\lib\site-packages\urllib3\util\connection.py", line 61, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "E:\python\anaconda3\envs\python_study\lib\socket.py", line 752, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 11004] getaddrinfo failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "E:\python\anaconda3\envs\python_study\lib\site-packages\urllib3\connectionpool.py", line 672, in urlopen
    chunked=chunked,
  File "E:\python\anaconda3\envs\python_study\lib\site-packages\urllib3\connectionpool.py", line 376, in _make_requ

In [40]:
def random_headers():
    headers_list = [{'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0'},{'User-agent':' Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'},{'User-agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.106 Safari/537.36 OPR/38.0.2220.41'},{'User-agent':'Mozilla/5.0 (Linux; U; Android 4.0.3; de-ch; HTC Sensation Build/IML74K) AppleWebKit/534.30 (KHTML, like Gecko) Version/4.0 Mobile Safari/534.30'},{'User-agent':'Mozilla/5.0 (compatible; MSIE 9.0; Windows Phone OS 7.5; Trident/5.0; IEMobile/9.0)'}]
    headers=random.choice(headers_list)
    print(headers)
    return headers

random_headers()


#從首頁開始爬是https://cookpad.com/tw after 後參數為linux_time 在轉換為現在時間
#while暫停條件


{'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0'}


{'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0'}

In [35]:
print(len(your_section))
print(your_section1)
print(your_section[:8])
print(your_section[8:16])
print(your_section[16:24])
print(your_section[24:32])
print(your_section[32:40])
print(your_section[40:46])

45
['/categories/6', '/categories/301', '/categories/302', '/categories/3', '/categories/40', '/categories/38/categories/16', '/categories/18']
['/categories/600', '/categories/614', '/categories/417', '/categories/25', '/categories/456', '/categories/459', '/categories/613', '/categories/219']
['/categories/185', '/categories/607', '/categories/30', '/categories/350', '/categories/498', '/categories/616', '/categories/52', '/categories/64']
['/categories/462', '/categories/599', '/categories/137', '/categories/63', '/categories/210', '/categories/211', '/categories/458', '/categories/49']
['/categories/71', '/categories/453', '/categories/13', '/categories/110', '/categories/602', '/categories/77', '/categories/20', '/categories/463']
['/categories/455', '/categories/495', '/categories/216', '/categories/449', '/categories/148', '/categories/50', '/categories/26', '/categories/29']
['/categories/147', '/categories/206', '/categories/39', '/categories/612', '/categories/27']


In [ ]:
['/categories/6', '/categories/301', '/categories/302', '/categories/3', '/categories/40', '/categories/38''/categories/16', '/categories/18']